<a href="https://colab.research.google.com/github/kekubhai/VIT-01/blob/main/VIt_model_onCFIR_10_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
torch.__version__

'2.6.0+cu124'

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import numpy as np
import random
import matplotlib.pyplot as plt


In [11]:
#setting up device agnostic code
device='cuda' if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [6]:
torchvision.__version__

'0.21.0+cu124'

In [14]:
torch.cuda.is_available()

False

In [16]:
#Set the seed
torch.manual_seed(42)
random.seed(42)

In [ ]:
#setting the hyperparameters